<a id="plan"></a>

## Preprocessing of Microsoft GUIDE dataset

-  [Observe training data](#observetrdata)
-  [Missing values in training data](#misvaltrdata)
-  [Drop id columns/ill columns in training data](#dropidtrdata)
-  [Feature engineering in training data](#featureengtrdata)
-  [Downcasting training data](#downcasttrdata)
-  [Handling MitreTechniques in training data](#mitretrdata)
-  [Numericalizing categoricals, analysis on training data](#numantrdata)
-  [Same preprocessing for testing data](#samepreptestdata)

Microsoft GUIDE dataset challenges us to create a ML model which will be able to evaluate incidents as BP, FP, TP (BenignPositive, FalsePositive, TruePositive respectively). The dataset has already feature-engineered/human-made columns. The data itself is hierarchical: events form alerts, alerts form incidents. Each row is an event.<br></br>
[Bottom](#bottom)

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path
import src.transformLargeDF as tldf

In [2]:
%matplotlib inline

In [3]:
data_path = Path().resolve() / 'data'
df_train = pd.read_csv(data_path / 'GUIDE_Train.csv', low_memory=False)
df_test = pd.read_csv(data_path / 'GUIDE_Test.csv', low_memory=False)

In [4]:
df_train.shape

(9516837, 45)

<a id="observetrdata"></a>

### Observe training data

[Back to top](#plan)

In [5]:
df_train.head().T

,0,1,2,3,4
Id,180388628218,455266534868,1056561957389,1279900258736,214748368522
OrgId,0,88,809,92,148
IncidentId,612,326,58352,32992,4359
AlertId,123247,210035,712507,774301,188041
Timestamp,2024-06-04T06:05:15.000Z,2024-06-14T03:01:25.000Z,2024-06-13T04:52:55.000Z,2024-06-10T16:39:36.000Z,2024-06-15T01:08:07.000Z
DetectorId,7,58,423,2,9
AlertTitle,6,43,298,2,74
Category,InitialAccess,Exfiltration,InitialAccess,CommandAndControl,Execution
MitreTechniques,NaN,NaN,T1189,NaN,NaN
IncidentGrade,TruePositive,FalsePositive,FalsePositive,BenignPositive,TruePositive


<table>
<tr>
<th>Feature</th><th>Description</th>
</tr>
<tr><td>Id</td><td>Unique ID for each OrgId-IncidentId pair</td></tr><tr><td>OrgId</td><td>Organization identifier</td></tr><tr><td>IncidentId</td><td>Organizationally unique incident identifier</td></tr><tr><td>AlertId</td><td>Unique identifier for an alert</td></tr><tr><td>Timestamp</td><td>Time the alert was created</td></tr><tr><td>DetectorId</td><td>Unique ID for the alert generating detector</td></tr><tr><td>AlertTitle</td><td>Title of the alert</td></tr><tr><td>Category</td><td>Category of the alert</td></tr><tr><td>MitreTechniques</td><td>MITRE ATT&CK techniques involved in alert</td></tr><tr><td>IncidentGrade</td><td>SOC grade assigned to the incident</td></tr><tr><td>ActionGrouped</td><td>SOC alert remediation action (high level)</td></tr><tr><td>ActionGranular</td><td>SOC alert remediation action (fine-grain)</td></tr><tr><td>EntityType</td><td>Type of entity involved in the alert</td></tr><tr><td>EvidenceRole</td><td>Role of the evidence in the investigation</td></tr><tr><td>Roles</td><td>Additional metadata on evidence role in alert</td></tr><tr><td>DeviceId</td><td>Unique identifier for the device</td></tr><tr><td>DeviceName</td><td>Name of the device</td></tr><tr><td>Sha256</td><td>SHA-256 hash of the file</td></tr><tr><td>IpAddress</td><td>IP address involved</td></tr><tr><td>Url</td><td>URL involved</td></tr><tr><td>AccountSid</td><td>On-premises account identifier</td></tr><tr><td>AccountUpn</td><td>Email account identifier</td></tr><tr><td>AccountObjectId</td><td>Entra ID account identifier</td></tr><tr><td>AccountName</td><td>Name of the on-premises account</td></tr><tr><td>NetworkMessageId</td><td>Org-level identifier for email message</td></tr><tr><td>EmailClusterId</td><td>Unique identifier for the email cluster</td></tr><tr><td>RegistryKey</td><td>Registry key involved</td></tr><tr><td>RegistryValueName</td><td>Name of the registry value</td></tr><tr><td>RegistryValueData</td><td>Data of the registry value</td></tr><tr><td>ApplicationId</td><td>Unique identifier for the application</td></tr><tr><td>ApplicationName</td><td>Name of the application</td></tr><tr><td>OAuthApplicationId</td><td>OAuth application identifier</td></tr><tr><td>ThreatFamily</td><td>Malware family associated with a file</td></tr><tr><td>FileName</td><td>Name of the file</td></tr><tr><td>FolderPath</td><td>Path of the file folder</td></tr><tr><td>ResourceIdName</td><td>Name of the Azure resource</td></tr><tr><td>ResourceType</td><td>Type of Azure resource</td></tr><tr><td>OSFamily</td><td>Family of the operating system</td></tr><tr><td>OSVersion</td><td>Version of the operating system</td></tr><tr><td>AntispamDirection</td><td>Direction of the antispam filter</td></tr><tr><td>SuspicionLevel</td><td>Level of suspicion</td></tr><tr><td>LastVerdict</td><td>Final verdict of threat analysis</td></tr><tr><td>CountryCode</td><td>Country code evidence appears in</td></tr><tr><td>State</td><td>State of evidence appears in</td></tr><tr><td>City</td><td>City evidence appears in</td></tr>
</table>

During observing data and reading [Microsoft scientific paper on it](https://arxiv.org/abs/2407.09017) I have found out that there is an extra column in the test dataset, let`s see it and remove:

In [6]:
f'Train has {len(df_train.columns)} columns, while test has {len(df_test.columns)} columns'

'Train has 45 columns, while test has 46 columns'

In [7]:
col_to_drop = df_test.columns.difference(df_train.columns)[0]
print('Dropped column:', df_test[col_to_drop], sep='\n')
df_test.drop(labels=['Usage'], axis=1, inplace=True)

Dropped column:
0          Private
1           Public
2           Public
3           Public
4           Public
            ...   
4147987     Public
4147988    Private
4147989     Public
4147990    Private
4147991     Public
Name: Usage, Length: 4147992, dtype: object


In [8]:
df_train.duplicated().sum()

np.int64(22559)

In [9]:
df_train.drop_duplicates(inplace=True)

<a id="misvaltrdata"></a>

### Missing values in training data

[Back to top](#plan)

In [10]:
mv_cols = df_train.isnull().sum()[df_train.isnull().sum() > 0]
mv_cols

MitreTechniques      5459368
IncidentGrade          51322
ActionGrouped        9438232
ActionGranular       9438232
EmailClusterId       9397469
ThreatFamily         9420765
ResourceType         9487219
Roles                9276141
AntispamDirection    9316981
SuspicionLevel       8052545
LastVerdict          7262442
dtype: int64

In [11]:
mv_cols / len(df_train)

MitreTechniques      0.575017
IncidentGrade        0.005406
ActionGrouped        0.994097
ActionGranular       0.994097
EmailClusterId       0.989803
ThreatFamily         0.992257
ResourceType         0.999256
Roles                0.977024
AntispamDirection    0.981326
SuspicionLevel       0.848147
LastVerdict          0.764928
dtype: float64

In [12]:
df_train[mv_cols.axes[0]]

,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EmailClusterId,ThreatFamily,ResourceType,Roles,AntispamDirection,SuspicionLevel,LastVerdict
0,NaN,TruePositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,FalsePositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T1189,FalsePositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
3,NaN,BenignPositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
4,NaN,TruePositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9516832,NaN,TruePositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9516833,NaN,BenignPositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
9516834,NaN,BenignPositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9516835,NaN,BenignPositive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here I encountered the first problem: some columns have an enormous amount of missing values, and, what is more important, our target has 0.5% data missing. Knowing that some of the columns are clearly made by SOC-analysts, I was left with two tactics:
-  Observe columns that are made by human in post-analysis of the incident, search for dependencies in decisions made, insert into missing values classes according to the probabilities of incident being BP, FP, TP having values X in N columns
-  LastVerdict seems to be the final decision made by a SOC-analyst. Insert classes into missing values where LastVerdict exists, drop all other rows<br></br>
I have chosen the second tactic as there is no probability to insert a wrong class, therefore harm the future model

In [13]:
df_train['LastVerdict'].value_counts()

LastVerdict
Suspicious                                            1401964
Malicious                                              431980
NoThreatsFound                                         397701
DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66        128
DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2         63
Name: count, dtype: int64

In [14]:
df_train[df_train['LastVerdict'] == 'DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66']['IncidentGrade'].value_counts()

IncidentGrade
TruePositive      72
BenignPositive    42
FalsePositive     14
Name: count, dtype: int64

In [15]:
df_train[df_train['LastVerdict'] == 'Suspicious']['IncidentGrade'].value_counts().sum()

np.int64(1396197)

In [16]:
df_train[df_train['LastVerdict'] == 'DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2']['IncidentGrade'].value_counts()

IncidentGrade
TruePositive    63
Name: count, dtype: int64

In [17]:
mapping = {
    'Malicious': 'TruePositive',
    'Suspicious': 'BenignPositive',
    'NoThreatsFound': 'FalsePositive',
}

df_train['IncidentGrade'] = df_train['IncidentGrade'].fillna(
    df_train['LastVerdict'].map(mapping)
)

In [18]:
df_train.dropna(subset=['IncidentGrade'], inplace=True)

<a id="dropidtrdata"></a>

### Drop some of id columns or ones that can badly affect training
Here I decided to drop some of the columns for the following reason: some of them contain too specific information about events occurred, for example, NetworkMessageId, which basically has a relation only to a particular event/incident and cannot be generalized. Moreover, some columns, like LastVerdict, contain post-event information and show the decision of a SOC-analyst regarding an event. Therefore, training a model on this column may result into pre-determined results in testing.<br></br>
[Back to top](#plan)

In [19]:
cols_to_leave = '''ThreatFamily
RegistryValueName
ApplicationName
AccountName
AlertTitle
EntityType
Category
MitreTechniques
IncidentGrade
OrgId
Timestamp
DetectorId
OSFamily
OSVersion
AntispamDirection
CountryCode
State
City'''.split('\n')

df_train.columns.difference(cols_to_leave).to_list()

['AccountObjectId',
 'AccountSid',
 'AccountUpn',
 'ActionGranular',
 'ActionGrouped',
 'AlertId',
 'ApplicationId',
 'DeviceId',
 'DeviceName',
 'EmailClusterId',
 'EvidenceRole',
 'FileName',
 'FolderPath',
 'Id',
 'IncidentId',
 'IpAddress',
 'LastVerdict',
 'NetworkMessageId',
 'OAuthApplicationId',
 'RegistryKey',
 'RegistryValueData',
 'ResourceIdName',
 'ResourceType',
 'Roles',
 'Sha256',
 'SuspicionLevel',
 'Url']

In [20]:
df_train = df_train[cols_to_leave]

Let us see the rest missing values among columns we have left

In [21]:
mv_cols = df_train.isnull().sum()[df_train.isnull().sum() > 0]
mv_cols

ThreatFamily         9376881
MitreTechniques      5420922
AntispamDirection    9273097
dtype: int64

In [22]:
mv_cols / len(df_train)

ThreatFamily         0.992221
MitreTechniques      0.573619
AntispamDirection    0.981239
dtype: float64

In [23]:
df_train['ThreatFamily'].unique()

array([nan, 'CymRan', 'CVE-2017-11882', ..., 'MedusaLocker', 'Cain',
       'Gootkit'], shape=(1746,), dtype=object)

In [24]:
df_train['MitreTechniques'].unique()

array([nan, 'T1189', 'T1078;T1078.004', ..., 'T0831;T0858',
       'T1055;T1055.002;T1055.004;T1055.012;T1071;T1106',
       'T1566;T0865;T1650'], shape=(1186,), dtype=object)

In [25]:
df_train['AntispamDirection'].unique()

array([nan, 'Inbound', 'Intraorg', 'Outbound',
       'DomainPII_df80ab894e01e375bf55d12ba315c04029d3e32d',
       'DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66'], dtype=object)

All missing values are in categorical features, and there are a lot of them. Decision: fill missing values with class 'Unknown', adding an information column that these classes were missing

In [26]:
df_train['AntispamDirection' + '_na'] = pd.isnull(df_train['AntispamDirection'])
df_train['AntispamDirection'] = df_train['AntispamDirection'].fillna('Unknown')

df_train['ThreatFamily' + '_na'] = pd.isnull(df_train['ThreatFamily'])
df_train['ThreatFamily'] = df_train['ThreatFamily'].fillna('NoFamily')

df_train['MitreTechniques' + '_na'] = pd.isnull(df_train['MitreTechniques'])
df_train['MitreTechniques'] = df_train['MitreTechniques'].fillna('Unknown')

In [27]:
df_train.isnull().sum()[df_train.isnull().sum() > 0]

Series([], dtype: int64)

<a id="featureengtrdata"></a>

### Feature engineering in training data
[Back to top](#plan)

In [28]:
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Timestamp'].sort_values()

4426918   2023-11-20 16:32:22+00:00
8310041   2023-11-22 10:08:16+00:00
4694615   2023-11-22 14:28:08+00:00
2603652   2023-11-23 14:58:14+00:00
6067124   2023-11-27 21:42:06+00:00
                     ...           
4371886   2024-06-17 14:34:51+00:00
965199    2024-06-17 14:45:38+00:00
6297130   2024-06-17 14:45:38+00:00
4007765   2024-06-17 14:45:38+00:00
7643048   2024-06-17 14:45:38+00:00
Name: Timestamp, Length: 9450394, dtype: datetime64[ns, UTC]

In [29]:
tldf.add_date_columns(df_train, 'Timestamp')

100%|██████████| 1/1 [00:02<00:00,  2.91s/it]


In [30]:
df_train.shape

(9450394, 34)

In [31]:
tldf.changeToCats(df_train)

34it [00:03, 10.38it/s]


In [32]:
df_train.dtypes[df_train.dtypes == 'category']

ThreatFamily          category
EntityType            category
Category              category
MitreTechniques       category
IncidentGrade         category
AntispamDirection     category
TimestampPartOfDay    category
dtype: object

In [33]:
df_train['TimestampPartOfDay'].cat.categories

Index(['Afternoon', 'Evening', 'Morning', 'Night'], dtype='object')

In [34]:
df_train['IncidentGrade'].cat.categories

Index(['BenignPositive', 'FalsePositive', 'TruePositive'], dtype='object')

In [35]:
df_train.IncidentGrade = df_train.IncidentGrade.cat.set_categories(['FalsePositive', 'BenignPositive', 'TruePositive'], ordered = True)

<a id="downcasttrdata"></a>

### Downcasting training data
[Back to top](#plan)

In [36]:
df_train.dtypes[df_train.dtypes == 'int64']

RegistryValueName    int64
ApplicationName      int64
AccountName          int64
AlertTitle           int64
OrgId                int64
DetectorId           int64
OSFamily             int64
OSVersion            int64
CountryCode          int64
State                int64
City                 int64
dtype: object

In [37]:
cols_int64 = df_train.dtypes[df_train.dtypes == 'int64'].index
possible_integers = pd.Series([8, 16, 32, 64])
for i in cols_int64:
  value_max_log = np.log2(df_train[i].max())
  value_min_log = np.log2(abs(df_train[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_train[i] = df_train[i].astype(f'int{transform_to.min()}')

In [38]:
df_train.dtypes[df_train.dtypes == 'int64']

Series([], dtype: object)

In [39]:
df_train.dtypes[df_train.dtypes == 'int32']

AccountName           int32
AlertTitle            int32
TimestampYear         int32
TimestampMonth        int32
TimestampDay          int32
TimestampDayofweek    int32
TimestampDayofyear    int32
TimestampHour         int32
dtype: object

In [40]:
cols_int32 = df_train.dtypes[df_train.dtypes == 'int32'].index
possible_integers = pd.Series([8, 16, 32])
for i in cols_int32:
  value_max_log = np.log2(df_train[i].max())
  value_min_log = np.log2(abs(df_train[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_train[i] = df_train[i].astype(f'int{transform_to.min()}')

In [41]:
df_train.dtypes[df_train.dtypes == 'int32']

AccountName    int32
AlertTitle     int32
dtype: object

In [42]:
df_train.to_feather(data_path / 'ready2eda.feather')

<a id="mitretrdata"></a>

### Handling MitreTechniques in training data
At the beginning, MitreTechniques was supposed to be multi-label binarized, all other categorical columns were supposed to be one-hot encoded, as well as the numerical columns. All this would have served a purpose to group by IncidentId and then aggregate rows to get incidents (data is hierarchical), so I would train the model directly on incidents. After reconsideration, I changed my mind on this. MitreTechniques and all other columns are too big in unique values, which would just blow the memory up (In Microsoft scientific paper about how they developed Copilot Guided Response it is written that they use OneHotEncoder followed by aggregation by incident id to get data prepared for ML). That said, I will have to change my strategy, as I cannot stick to this one in terms of computing power. Below is feature engineering upon  MitreTechniques column, as some of entries have multiple labels separated by ;. Training of the model will be done on cyber events instead of incidents.<br></br>
[Back to top](#plan)

In [43]:
ser = df_train['MitreTechniques']
tags_lists = ser.str.split(';').map(lambda lst: [t for t in lst if t] if isinstance(lst, list) else [])

counter = Counter()

for lst in tags_lists:
    for t in lst:
        counter[t] += 1

top_tags = [t for t,_ in counter.most_common()]
tag_to_idx = {t:i for i,t in enumerate(top_tags)}


exploded = tags_lists.explode().rename('tag').to_frame()
exploded = exploded[exploded['tag'].notna()]
exploded = exploded[exploded['tag'] != '']


exploded['tag_freq'] = exploded['tag'].map(counter)


agg_grouped = exploded.groupby(exploded.index).agg(
    mitre_tag_count=('tag', 'count'),
    mitre_tags_freq_sum=('tag_freq', 'sum')
)


top_tag = exploded.sort_values(['tag_freq'], ascending=False).groupby(exploded.index).first()['tag']
agg_grouped['mitre_top_tag_code'] = top_tag.map(lambda t: tag_to_idx.get(t, -1))


agg_grouped = agg_grouped.reindex(df_train.index, fill_value=0)
agg_grouped['mitre_has_multiple'] = agg_grouped['mitre_tag_count'] > 1


agg_grouped['mitre_tag_count'] = agg_grouped['mitre_tag_count'].astype('int32')
agg_grouped['mitre_has_multiple'] = agg_grouped['mitre_has_multiple'].astype('bool')
agg_grouped['mitre_top_tag_code'] = agg_grouped['mitre_top_tag_code'].astype('int32')
agg_grouped['mitre_tags_freq_sum'] = agg_grouped['mitre_tags_freq_sum'].astype('int32')

df_train = pd.concat([df_train, agg_grouped], axis=1)

In [44]:
cols_int32 = df_train.dtypes[df_train.dtypes == 'int32'].index
possible_integers = pd.Series([8, 16, 32])
for i in cols_int64:
  value_max_log = np.log2(df_train[i].max())
  value_min_log = np.log2(abs(df_train[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_train[i] = df_train[i].astype(f'int{transform_to.min()}')

C:\Users\korot\AppData\Local\Temp\ipykernel_1220\279177163.py:5: RuntimeWarning: overflow encountered in scalar absolute
  value_min_log = np.log2(abs(df_train[i].min()) + 1e-6)
C:\Users\korot\AppData\Local\Temp\ipykernel_1220\279177163.py:5: RuntimeWarning: invalid value encountered in log2
  value_min_log = np.log2(abs(df_train[i].min()) + 1e-6)


<a id="numantrdata"></a>

### Numericalizing categoricals, analysis on the columns in training data
[Back to top](#plan)

In [45]:
categorical_cols = df_train.dtypes[df_train.dtypes == 'category'].index

for i in categorical_cols:
  df_train[i] = df_train[i].cat.codes

In [46]:
df_train.head()

,ThreatFamily,RegistryValueName,ApplicationName,AccountName,AlertTitle,EntityType,Category,MitreTechniques,IncidentGrade,OrgId,...,TimestampIs_quarter_end,TimestampIs_year_start,TimestampIs_year_end,TimestampHour,TimestampIsWeekend,TimestampPartOfDay,mitre_tag_count,mitre_tags_freq_sum,mitre_top_tag_code,mitre_has_multiple
0,1042,635,3421,453297,6,15,10,1185,2,0,...,False,False,False,6,False,2,1,5420922,0,False
1,1042,635,3421,24887,43,32,7,1185,0,88,...,False,False,False,3,False,3,1,5420922,0,False
2,1042,635,3421,453297,298,31,10,859,0,809,...,False,False,False,4,False,3,1,8984,0,False
3,1042,635,3421,453297,2,31,1,1185,1,92,...,False,False,False,16,False,0,1,5420922,0,False
4,1042,635,3421,479,74,32,6,1185,2,148,...,False,False,False,1,True,3,1,5420922,0,False


In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9450394 entries, 0 to 9516836
Data columns (total 38 columns):
 #   Column                     Dtype
---  ------                     -----
 0   ThreatFamily               int16
 1   RegistryValueName          int16
 2   ApplicationName            int16
 3   AccountName                int32
 4   AlertTitle                 int32
 5   EntityType                 int8 
 6   Category                   int8 
 7   MitreTechniques            int16
 8   IncidentGrade              int8 
 9   OrgId                      int16
 10  DetectorId                 int16
 11  OSFamily                   int8 
 12  OSVersion                  int8 
 13  AntispamDirection          int8 
 14  CountryCode                int8 
 15  State                      int16
 16  City                       int16
 17  AntispamDirection_na       bool 
 18  ThreatFamily_na            bool 
 19  MitreTechniques_na         bool 
 20  TimestampYear              int16
 21  TimestampMont

In [48]:
numeric_df = df_train.select_dtypes(include='number')
correlation_matrix = numeric_df.corr().abs()


high_corr_pairs = (
    correlation_matrix.where((correlation_matrix > 0.7) & (correlation_matrix < 1))
    .stack()
    .reset_index()
    .rename(columns={'level_0': 'Feature1', 'level_1': 'Feature2', 0: 'Correlation'})
    .sort_values(by='Correlation', ascending=False)
)

print("Highly correlated column pairs with correlation greater than 0.7:")
print(high_corr_pairs)

Highly correlated column pairs with correlation greater than 0.7:
              Feature1            Feature2  Correlation
0             OSFamily           OSVersion     0.999289
1            OSVersion            OSFamily     0.999289
5                State                City     0.996487
7                 City               State     0.996487
8       TimestampMonth        TimestampDay     0.779586
10        TimestampDay      TimestampMonth     0.779586
13  TimestampPartOfDay       TimestampHour     0.754387
12       TimestampHour  TimestampPartOfDay     0.754387
3          CountryCode                City     0.737930
6                 City         CountryCode     0.737930
9       TimestampMonth  TimestampDayofyear     0.735941
11  TimestampDayofyear      TimestampMonth     0.735941
4                State         CountryCode     0.728805
2          CountryCode               State     0.728805


In [49]:
df_train.to_feather(data_path / 'ready2model.feather')

<a id="samepreptestdata"></a>

### Preprocessing testing data
[Back to top](#plan)

In [50]:
df_test.shape

(4147992, 45)

In [51]:
df_test.head().T

,0,1,2,3,4
Id,1245540519230,1400159342154,1279900255923,60129547292,515396080539
OrgId,657,3,145,222,363
IncidentId,11767,91158,32247,15294,7615
AlertId,87199,632273,131719,917686,5944
Timestamp,2024-06-04T22:56:27.000Z,2024-06-03T12:58:26.000Z,2024-06-08T03:20:49.000Z,2024-06-12T12:07:31.000Z,2024-06-06T17:42:05.000Z
DetectorId,524,2,2932,0,27
AlertTitle,563,2,10807,0,18
Category,LateralMovement,CommandAndControl,LateralMovement,InitialAccess,Discovery
MitreTechniques,T1021;T1047;T1105;T1569.002,NaN,T1021;T1027.002;T1027.005;T1105,T1078;T1078.004,T1087;T1087.002
IncidentGrade,BenignPositive,BenignPositive,BenignPositive,FalsePositive,BenignPositive


In [52]:
df_test.duplicated().sum()

np.int64(104)

In [53]:
df_test.drop_duplicates(inplace=True)

In [54]:
mv_cols = df_test.isnull().sum()[df_test.isnull().sum() > 0]
mv_cols

MitreTechniques      2307084
ActionGrouped        4145975
ActionGranular       4145975
EmailClusterId       4106182
ThreatFamily         4116518
ResourceType         4144894
Roles                4039214
AntispamDirection    4071379
SuspicionLevel       3498101
LastVerdict          3155218
dtype: int64

In [55]:
mv_cols / len(df_test)

MitreTechniques      0.556207
ActionGrouped        0.999539
ActionGranular       0.999539
EmailClusterId       0.989945
ThreatFamily         0.992437
ResourceType         0.999278
Roles                0.973800
AntispamDirection    0.981555
SuspicionLevel       0.843345
LastVerdict          0.760681
dtype: float64

In [56]:
df_test[mv_cols.axes[0]]

,MitreTechniques,ActionGrouped,ActionGranular,EmailClusterId,ThreatFamily,ResourceType,Roles,AntispamDirection,SuspicionLevel,LastVerdict
0,T1021;T1047;T1105;T1569.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
2,T1021;T1027.002;T1027.005;T1105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspicious,Suspicious
3,T1078;T1078.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T1087;T1087.002,NaN,NaN,NaN,NaN,NaN,Suspicious,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4147987,T1566.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4147988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4147989,T1046;T1071;T1210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4147990,T1566.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
cols_to_leave = '''ThreatFamily
RegistryValueName
ApplicationName
AccountName
AlertTitle
EntityType
Category
MitreTechniques
IncidentGrade
OrgId
Timestamp
DetectorId
OSFamily
OSVersion
AntispamDirection
CountryCode
State
City'''.split('\n')

df_test.columns.difference(cols_to_leave).to_list()

['AccountObjectId',
 'AccountSid',
 'AccountUpn',
 'ActionGranular',
 'ActionGrouped',
 'AlertId',
 'ApplicationId',
 'DeviceId',
 'DeviceName',
 'EmailClusterId',
 'EvidenceRole',
 'FileName',
 'FolderPath',
 'Id',
 'IncidentId',
 'IpAddress',
 'LastVerdict',
 'NetworkMessageId',
 'OAuthApplicationId',
 'RegistryKey',
 'RegistryValueData',
 'ResourceIdName',
 'ResourceType',
 'Roles',
 'Sha256',
 'SuspicionLevel',
 'Url']

In [58]:
df_test = df_test[cols_to_leave]

In [59]:
mv_cols = df_test.isnull().sum()[df_test.isnull().sum() > 0]
mv_cols

ThreatFamily         4116518
MitreTechniques      2307084
AntispamDirection    4071379
dtype: int64

In [60]:
mv_cols / len(df_test)

ThreatFamily         0.992437
MitreTechniques      0.556207
AntispamDirection    0.981555
dtype: float64

In [61]:
df_test['AntispamDirection' + '_na'] = pd.isnull(df_test['AntispamDirection'])
df_test['AntispamDirection'] = df_test['AntispamDirection'].fillna('Unknown')

df_test['ThreatFamily' + '_na'] = pd.isnull(df_test['ThreatFamily'])
df_test['ThreatFamily'] = df_test['ThreatFamily'].fillna('NoFamily')

df_test['MitreTechniques' + '_na'] = pd.isnull(df_test['MitreTechniques'])
df_test['MitreTechniques'] = df_test['MitreTechniques'].fillna('Unknown')

In [62]:
df_test.isnull().sum()[df_test.isnull().sum() > 0]

Series([], dtype: int64)

In [63]:
df_test.head().T

,0,1,2,3,4
ThreatFamily,NoFamily,NoFamily,NoFamily,NoFamily,NoFamily
RegistryValueName,635,635,635,635,635
ApplicationName,3421,3421,3421,3421,3421
AccountName,863,453297,453297,453297,136104
AlertTitle,563,2,10807,0,18
EntityType,User,Machine,Process,CloudLogonSession,User
Category,LateralMovement,CommandAndControl,LateralMovement,InitialAccess,Discovery
MitreTechniques,T1021;T1047;T1105;T1569.002,Unknown,T1021;T1027.002;T1027.005;T1105,T1078;T1078.004,T1087;T1087.002
IncidentGrade,BenignPositive,BenignPositive,BenignPositive,FalsePositive,BenignPositive
OrgId,657,3,145,222,363


In [64]:
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Timestamp'].sort_values()

1900748   2023-11-22 15:55:05+00:00
1068945   2023-11-28 17:03:39+00:00
297259    2023-11-29 16:05:48+00:00
1126304   2023-12-02 02:55:54+00:00
458617    2023-12-02 13:39:13+00:00
                     ...           
1874011   2024-06-17 13:30:09+00:00
1556361   2024-06-17 13:30:09+00:00
1676256   2024-06-17 13:30:09+00:00
1315098   2024-06-17 13:30:09+00:00
84909     2024-06-17 13:30:09+00:00
Name: Timestamp, Length: 4147888, dtype: datetime64[ns, UTC]

In [65]:
tldf.add_date_columns(df_test, 'Timestamp')

100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


In [66]:
df_test.shape

(4147888, 34)

In [67]:
tldf.changeToCats(df_test)

34it [00:01, 23.49it/s]


In [68]:
df_test.dtypes[df_test.dtypes == 'category']

ThreatFamily          category
EntityType            category
Category              category
MitreTechniques       category
IncidentGrade         category
AntispamDirection     category
TimestampPartOfDay    category
dtype: object

In [69]:
df_test['TimestampPartOfDay'].cat.categories

Index(['Afternoon', 'Evening', 'Morning', 'Night'], dtype='object')

In [70]:
df_test['IncidentGrade'].cat.categories

Index(['BenignPositive', 'FalsePositive', 'TruePositive'], dtype='object')

In [71]:
df_test.IncidentGrade = df_test.IncidentGrade.cat.set_categories(['FalsePositive', 'BenignPositive', 'TruePositive'], ordered = True)

In [72]:
df_test.dtypes[df_test.dtypes == 'int64']

RegistryValueName    int64
ApplicationName      int64
AccountName          int64
AlertTitle           int64
OrgId                int64
DetectorId           int64
OSFamily             int64
OSVersion            int64
CountryCode          int64
State                int64
City                 int64
dtype: object

In [73]:
cols_int64 = df_test.dtypes[df_test.dtypes == 'int64'].index
possible_integers = pd.Series([8, 16, 32, 64])
for i in cols_int64:
  value_max_log = np.log2(df_test[i].max())
  value_min_log = np.log2(abs(df_test[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_test[i] = df_test[i].astype(f'int{transform_to.min()}')

In [74]:
df_test.dtypes[df_test.dtypes == 'int64']

Series([], dtype: object)

In [75]:
df_test.dtypes[df_test.dtypes == 'int32']

AccountName           int32
AlertTitle            int32
TimestampYear         int32
TimestampMonth        int32
TimestampDay          int32
TimestampDayofweek    int32
TimestampDayofyear    int32
TimestampHour         int32
dtype: object

In [76]:
cols_int32 = df_test.dtypes[df_test.dtypes == 'int32'].index
possible_integers = pd.Series([8, 16, 32])
for i in cols_int32:
  value_max_log = np.log2(df_test[i].max())
  value_min_log = np.log2(abs(df_test[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_test[i] = df_test[i].astype(f'int{transform_to.min()}')

In [77]:
df_test.dtypes[df_test.dtypes == 'int32']

AccountName    int32
AlertTitle     int32
dtype: object

In [78]:
ser = df_test['MitreTechniques']
tags_lists = ser.str.split(';').map(lambda lst: [t for t in lst if t] if isinstance(lst, list) else [])

counter = Counter()

for lst in tags_lists:
    for t in lst:
        counter[t] += 1

top_tags = [t for t,_ in counter.most_common()]
tag_to_idx = {t:i for i,t in enumerate(top_tags)}


exploded = tags_lists.explode().rename('tag').to_frame()
exploded = exploded[exploded['tag'].notna()]
exploded = exploded[exploded['tag'] != '']


exploded['tag_freq'] = exploded['tag'].map(counter)


agg_grouped = exploded.groupby(exploded.index).agg(
    mitre_tag_count=('tag', 'count'),
    mitre_tags_freq_sum=('tag_freq', 'sum')
)


top_tag = exploded.sort_values(['tag_freq'], ascending=False).groupby(exploded.index).first()['tag']
agg_grouped['mitre_top_tag_code'] = top_tag.map(lambda t: tag_to_idx.get(t, -1))


agg_grouped = agg_grouped.reindex(df_test.index, fill_value=0)
agg_grouped['mitre_has_multiple'] = agg_grouped['mitre_tag_count'] > 1


agg_grouped['mitre_tag_count'] = agg_grouped['mitre_tag_count'].astype('int32')
agg_grouped['mitre_has_multiple'] = agg_grouped['mitre_has_multiple'].astype('bool')
agg_grouped['mitre_top_tag_code'] = agg_grouped['mitre_top_tag_code'].astype('int32')
agg_grouped['mitre_tags_freq_sum'] = agg_grouped['mitre_tags_freq_sum'].astype('int32')

df_test = pd.concat([df_test, agg_grouped], axis=1)

In [79]:
cols_int32 = df_test.dtypes[df_test.dtypes == 'int32'].index
possible_integers = pd.Series([8, 16, 32])
for i in cols_int64:
  value_max_log = np.log2(df_test[i].max())
  value_min_log = np.log2(abs(df_test[i].min()) + 1e-6)
  biggest_log = max(value_max_log, value_min_log)
  transform_to = possible_integers[possible_integers > biggest_log]
  df_test[i] = df_test[i].astype(f'int{transform_to.min()}')

C:\Users\korot\AppData\Local\Temp\ipykernel_1220\3515747314.py:5: RuntimeWarning: overflow encountered in scalar absolute
  value_min_log = np.log2(abs(df_test[i].min()) + 1e-6)
C:\Users\korot\AppData\Local\Temp\ipykernel_1220\3515747314.py:5: RuntimeWarning: invalid value encountered in log2
  value_min_log = np.log2(abs(df_test[i].min()) + 1e-6)


In [80]:
categorical_cols = df_test.dtypes[df_test.dtypes == 'category'].index

for i in categorical_cols:
  df_test[i] = df_test[i].cat.codes

In [81]:
df_test.head()

,ThreatFamily,RegistryValueName,ApplicationName,AccountName,AlertTitle,EntityType,Category,MitreTechniques,IncidentGrade,OrgId,...,TimestampIs_quarter_end,TimestampIs_year_start,TimestampIs_year_end,TimestampHour,TimestampIsWeekend,TimestampPartOfDay,mitre_tag_count,mitre_tags_freq_sum,mitre_top_tag_code,mitre_has_multiple
0,688,635,3421,863,563,28,11,311,1,657,...,False,False,False,22,False,1,4,41055,0,True
1,688,635,3421,453297,2,15,1,985,1,3,...,False,False,False,12,False,0,1,2307084,0,False
2,688,635,3421,453297,10807,23,11,303,1,145,...,False,False,False,3,True,3,4,74498,0,True
3,688,635,3421,453297,0,7,10,577,0,222,...,False,False,False,12,False,0,2,1276949,0,True
4,688,635,3421,136104,18,28,5,610,1,363,...,False,False,False,17,False,0,2,85684,0,True


In [82]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4147888 entries, 0 to 4147991
Data columns (total 38 columns):
 #   Column                     Dtype
---  ------                     -----
 0   ThreatFamily               int16
 1   RegistryValueName          int16
 2   ApplicationName            int16
 3   AccountName                int32
 4   AlertTitle                 int32
 5   EntityType                 int8 
 6   Category                   int8 
 7   MitreTechniques            int16
 8   IncidentGrade              int8 
 9   OrgId                      int16
 10  DetectorId                 int16
 11  OSFamily                   int8 
 12  OSVersion                  int8 
 13  AntispamDirection          int8 
 14  CountryCode                int8 
 15  State                      int16
 16  City                       int16
 17  AntispamDirection_na       bool 
 18  ThreatFamily_na            bool 
 19  MitreTechniques_na         bool 
 20  TimestampYear              int16
 21  TimestampMont

In [83]:
df_train.to_feather(data_path / 'test_ready2model.feather')

<a id="bottom"></a>
[Back to top](#plan)